## **AB TESTING**

AB TESTİ: İki farklı değişkenin arasında karşılaştırma yapılmak istenildiğinde kullanılır.

* Ortalama ve oran karşılaştırıldığında kullanılır.

* A ve B ifadeleri kontrol grubu ve deney grubu anlamına gelir. 

* Test sonucundan elde edilen p değeri 0.05'den küçük olduğu zaman H0 iptal ediyoruz.


**Varsayımlar**: 
* Normallik varsayımı: İki dağılımın ağılımların normal olduğu varsayımı
* Varyans homojenliği: İki grubun dağılımlarının benzer olup olmaması

**Süreç** 

1 - Hipotezi Kur

2 - Varsayımları incele

3 - Pvalue değerine göre yorumla

In [29]:
## Bu çalışmada kullanacağımız kütüphanelerin import işlemini yapalım;
import itertools 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp,shapiro,levene,ttest_ind,mannwhitneyu 
from scipy.stats import pearsonr, spearmanr, kendalltau,f_oneway,kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",10)
pd.set_option("display.float_format", lambda x: "%.5f" %x)


--------

#### AB Testing (Bağımsız İki Örneklem T Testi)
Oranları ve ortalamaları test ediyor olduğumuzda kullanırız.

**SÜREÇ**
1. Hipotezleri Kur
2. Varsayım Kontrolü
    - 1. Normallik Varsayımı
    - 2. Varyans Homojenliği
3. Hipotezin Uygulanması
    - 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
    - 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)
4. p-value değerine göre sonuçları yorumla

*Not: Normallik sağlanmıyorsa direk 2 numara. Varyans homojenliği sağlanmıyorsa 1 numaraya arguman girilir.Normallik incelemesi öncesi aykırı değer incelemesi ve düzeltmesi yapmak faydalı olabilir.*



In [30]:
############################
# Uygulama 1: Sigara İçenler ile İçmeyenlerin Hesap Ortalamaları Arasında İst Ol An Fark var mı?
############################

df = sns.load_dataset("tips")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99000,1.01000,Female,No,Sun,Dinner,2
1,10.34000,1.66000,Male,No,Sun,Dinner,3
2,21.01000,3.50000,Male,No,Sun,Dinner,3
3,23.68000,3.31000,Male,No,Sun,Dinner,2
4,24.59000,3.61000,Female,No,Sun,Dinner,4


In [31]:
# sigara içenlerin hesap oranını hesaplayalım; 

df.groupby("smoker").agg({"total_bill":"mean"})

,total_bill
smoker,
Yes,20.75634
No,19.18828


In [32]:
# 1. Hipotezi Kur
# HO : M1 = M2
# H1 : M1 != M2

# 2. Varsayım Kontrolü
# Normallik Varsayımı
    #HO:Normallik sağlanmaktadır.
    #H1:Normallik sağlanmamaktadır.

# from scipy.stats import shapiro

test_stat,pvalue = shapiro(df.loc[df["smoker"] == "Yes","total_bill"])
print("Test Stat = %.4f, p-value = %.4f"% (test_stat,pvalue))

# p-value < ise 0.05'ten H0 RED
# p-value < değilse 0.05 H= REDDEDİLEMEZ

Test Stat = 0.9367, p-value = 0.0002


In [33]:
# Varyans Homojenliği Varsayımı

# HO: Varyanslar Homojendir.
# H1: Varyanslar Homojen değildir.

#from scipy.stats import levene

test_stat,pvalue = levene(df.loc[df["smoker"] == "Yes","total_bill"],df.loc[df["smoker"] == "No","total_bill"])
print("Test Stat = %.4f, p-value = %.4f"% (test_stat,pvalue))



Test Stat = 4.0537, p-value = 0.0452


In [34]:
# Hipotezin uygulanması

# 1.Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
# 2.Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)


In [35]:
## Durum varsayımların sağlandığı durum.
# varyans homojenliği sağlanmadığı zaman equal_var = False olur.

# from scipy stats import ttest_ind

test_stat,pvalue = ttest_ind(df.loc[df["smoker"] == "Yes","total_bill"],
                             df.loc[df["smoker"] == "No","total_bill"], equal_var=True)
print("Test Stat = %.4f, p-value = %.4f"% (test_stat,pvalue))

#! H0 RED.

Test Stat = 1.3384, p-value = 0.1820


In [36]:
## Durum varsayımların sağlanmadığı durum.
# nonparametrik ortlaama kıyasalama, medyan kıyasalama testi 

test_stat,pvalue = mannwhitneyu(df.loc[df["smoker"] == "Yes","total_bill"],
                             df.loc[df["smoker"] == "No","total_bill"])
print("Test Stat = %.4f, p-value = %.4f"% (test_stat,pvalue))


Test Stat = 7531.5000, p-value = 0.3413


In [37]:
# Uygulama 2: Titanic Kadın ve Erkek Yolcuların Yaş Ortalamaları Arasında İstatistiksel Olarak Anl. Fark. var mıdır?

# veri setinin import edilmesi

df = sns.load_dataset("titanic")

df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00000,1,0,7.25000,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00000,1,0,71.28330,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00000,0,0,7.92500,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00000,1,0,53.10000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00000,0,0,8.05000,S,Third,man,True,NaN,Southampton,no,True


In [38]:
# varsayım kontrolü 1.

# normallik varsayımı | shapiro

# h0 : normal dağılmıştır.
# h1 : normal dağılmamıştır.

df.dropna()
from scipy.stats import shapiro

t_test,p_value = shapiro(df.loc[df["sex"] == "female","age"].dropna())
print("t_test : %.4f\np_value :%.4f"%(t_test,p_value))

# ho red .
# normal dağılmamıştır.


t_test : 0.9848
p_value :0.0071


In [39]:
# varyans homojenliği varsayımı |

# ho: homojendir.
# h1: homojen değildir.

from scipy.stats import levene


t_stats,p_value = levene(df.loc[df["sex"] == "female","age"].dropna(),df.loc[df["sex"] == "male","age"].dropna())
print("t_stats: %.4f\np_value: %.4f" %(t_stats,p_value))

# h0 red edilemez 
# homojen dağılmıştır.

t_stats: 0.0013
p_value: 0.9712


In [40]:
# normallik varsayımı sağlansaydı

# hipotezlerin kurulması
# yolcuların cinsiyet ve yaş ortalaması hakkında istatiksel bir anlam farklılığı var mıdır ?


# H0 : Anlam farklılığı yoktur.
# H1 : Anlam farklılığı vardır.

from scipy.stats import ttest_ind



t_stats,p_value = ttest_ind(df.loc[df["sex"]=="female","age"].dropna(),df.loc[df["sex"]=="male","age"].dropna(),equal_var=True)

print("t_stats: %.4f\np_value: %.4f" %(t_stats,p_value))

# p < 0.05 bu sebepten h0 red
# Anlam farklılığı vardır. 

t_stats: -2.4992
p_value: 0.0127


In [41]:
# nanparametrik


from scipy.stats import mannwhitneyu

t_stats,p_value = mannwhitneyu(df.loc[df["sex"]=="female","age"].dropna(),df.loc[df["sex"]=="male","age"].dropna())
print("t_stats: %.4f\np_value: %.4f" %(t_stats,p_value))

# HO red 
# anlamlı bir farklılık vardır.

t_stats: 53212.5000
p_value: 0.0261


In [42]:
############################
# Uygulama 3: Diyabet Hastası Olan ve Olmayanların Yaşları Ort. Arasında İst. Ol. Anl. Fark var mıdır?
############################

# Veri setinin tanımlayalım;

df = pd.read_csv(r"C:\Users\kkakt\Desktop\Measurement Problems\datasets\diabetes.csv")

df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.60000,0.62700,50,1
1,1,85,66,29,0,26.60000,0.35100,31,0
2,8,183,64,0,0,23.30000,0.67200,32,1
3,1,89,66,23,94,28.10000,0.16700,21,0
4,0,137,40,35,168,43.10000,2.28800,33,1


In [43]:
# değişkenlerin istatiksel betimlenmesini inceleyelim;

df.groupby("Outcome").agg({"Age":"mean"})



,Age
Outcome,
0,31.19000
1,37.06716


In [44]:
# normallik varsayımı

#ho: dağılım normaldir.
#ho: dağılım normal değildir.

from scipy.stats import shapiro

t_test,p_value = shapiro(df.loc [df["Outcome"]== 0,"Age"])

print("0\nt_test: %.4f \np_value: %.4f" %(t_test,p_value))

t_test,p_value = shapiro(df.loc [df["Outcome"]== 1,"Age"])

print("1\nt_test: %.4f \np_value: %.4f" %(t_test,p_value))

# p<0.05 yani ho red.
# dağılım normal değil.


0
t_test: 0.8012 
p_value: 0.0000
1
t_test: 0.9546 
p_value: 0.0000


In [45]:
# normal dağılmadığı için gerek yok ama yine de; 
# homomjenlik varsayımı

# ho: homojendir.
# h1: homojen değildir.

from scipy.stats import levene

t_test,p_value = levene(df.loc [df["Outcome"]== 0,"Age"],df.loc [df["Outcome"]== 1,"Age"])

print("t_test: %.4f \np_value: %.4f" %(t_test,p_value))

# p>0.05 h0 red edilemez
# homojendir.


t_test: 2.2252 
p_value: 0.1362


In [46]:
#! Nonparametrik test uygulacağız (mannwitneyu)



# h0: m1 = m2   ==> anlamlı bir farklılık yoktur
# h1: m1 != m2  ==>anlamlı bir farklılık vardır.

from scipy.stats import mannwhitneyu

t_test,p_value = mannwhitneyu(df.loc [df["Outcome"]==1,"Age"],df.loc [df["Outcome"]==0,"Age"])

print("t_test: %.4f \np_value: %.4f" %(t_test,p_value))

# ho red edilir
# yaş değişkeni ile diyabet hastalığı arasında anlamlı bir farklılık vardır.



t_test: 92050.0000 
p_value: 0.0000


In [47]:
#! normal dağıldığını varsaydığımız senaryoda



# h0: m1 = m2   ==> anlamlı bir farklılık yoktur
# h1: m1 != m2  ==>anlamlı bir farklılık vardır.

from scipy.stats import ttest_ind

t_test,p_value = ttest_ind(df.loc [df["Outcome"]==1,"Age"],df.loc [df["Outcome"]==0,"Age"])

print("t_test: %.4f \np_value: %.4f" %(t_test,p_value))

# ho red edilir
# yaş değişkeni ile diyabet hastalığı arasında anlamlı bir farklılık vardır.

t_test: 6.7927 
p_value: 0.0000


In [48]:
###################################################
# İş Problemi: Kursun Büyük Çoğunluğunu İzleyenler ile İzlemeyenlerin Puanları Birbirinden Farklı mı?
###################################################


# H0: M1 = M2 (... iki grup ortalamaları arasında ist ol.anl.fark yoktur.)
# H1: M1 != M2 (...vardır)

df = pd.read_csv(r"C:\Users\kkakt\Desktop\Measurement Problems\datasets\course_reviews.csv")
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000
3,5.00000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.00000,0.00000,0.00000
4,4.00000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.00000,0.00000,0.00000


In [49]:
# ilgilenilen değişkenlerin istatiksel betimlemeleri

df.groupby("Rating").agg({"Progress": "mean"})

,Progress
Rating,
1.00000,42.86667
1.50000,4.50000
2.00000,14.08333
2.50000,31.18182
3.00000,17.83871
3.50000,17.64583
4.00000,17.88773
4.50000,23.01263
5.00000,29.33242


In [50]:
# normallik varsayımı

df[(df["Progress"] > 75)]["Rating"].mean()
df[(df["Progress"] > 25)]["Rating"].mean()

test_stat, pvalue = shapiro(df[(df["Progress"] > 75)]["Rating"])

print("t_test: %.4f \np_value: %.4f" %(t_test,p_value))

# ho red normal dağılmamıştır.


t_test: 6.7927 
p_value: 0.0000


In [51]:
# nonparametrik varsayımı

df[(df["Progress"] > 75)]["Rating"].mean()
df[(df["Progress"] > 25)]["Rating"].mean()



test_stat, pvalue = mannwhitneyu(df[(df["Progress"] > 75)]["Rating"],df[(df["Progress"] > 25)]["Rating"])

print("t_test: %.4f \np_value: %.4f" %(t_test,p_value))



# ilerleme ve puanlama arasında bir ilişki vardır.

t_test: 6.7927 
p_value: 0.0000


------

#### AB Testing (Bağımsız İki Örneklem T Testi)

varsayım: 
**n>30 olması gereklidir.**
    

In [52]:
# veri setini oluşturalım ve hipotezimizi kuralım

# iş problemi : eski tasarım ve yeni tasraım arasındaki tıklanma sayısı arasında bir ilişki var mı ? 

# H0: p1 = p2
# Yeni Tasarımın Dönüşüm Oranı ile Eski Tasarımın Dönüşüm Oranı Arasında İst. Ol. Anlamlı Farklılık Yoktur.
# H1: p1 != p2
# ... vardır

basari_sayisi = np.array([300, 250])
gozlem_sayilari = np.array([1000, 1100])

# array'lere koy ve gönder


In [53]:
# AB Oran testinin uygulanması

from statsmodels.stats.proportion import proportions_ztest

proportions_ztest(count=basari_sayisi,nobs=gozlem_sayilari)

# h0 red
# ilişki vardır.

(3.7857863233209255, 0.0001532232957772221)

In [54]:
############################
# Uygulama: Kadın ve Erkeklerin Hayatta Kalma Oranları Arasında İst. Olarak An. Farklılık var mıdır?
############################

# H0: p1 = p2
# Kadın ve Erkeklerin Hayatta Kalma Oranları Arasında İst. Olarak An. Fark yoktur

# H1: p1 != p2
# .. vardır

df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00000,1,0,7.25000,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00000,1,0,71.28330,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00000,0,0,7.92500,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00000,1,0,53.10000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00000,0,0,8.05000,S,Third,man,True,NaN,Southampton,no,True


In [55]:
# kadınların hayatta kalma oranı
df.loc[df["sex"] == "female","survived"].mean()

#erkeklerin hayatta kalma oranı
df.loc[df["sex"]=="male","survived"].mean()


0.18890814558058924

In [56]:
# testin uygulanması
# proportions fonksiyonunun bizden beklediği başarı sayısı ve toplam gözlem sayısı

from statsmodels.stats.proportion import proportions_ztest

# başarı sayıları:

female_succ_count = df.loc [df["sex"]=="female","survived"].sum()
male_succ_count = df.loc [df["sex"]=="male","survived"].sum()

proportions_ztest(count=[female_succ_count,male_succ_count], 
                  nobs = [df.loc [df["sex"]=="female","survived"].shape[0],df.loc [df["sex"]=="male","survived"].shape[0]])

print("t_test: %.4f \np_value: %.4f" %(t_test,p_value))

# h0 red 
# anlamlı bir farklılık vardır.

t_test: 6.7927 
p_value: 0.0000


----------

#### İkiden Fazla Grup Ortalaması Karşılaştırma ( ANOVA-Analysis of Variance )

Anova analizi birden fazla grup arasında farkın olup olmadığını inceler.

Varsayımlar:

- nomrallik varsayımı
- homojen dağılımı varsayımı

varsayımlar sağlanıyorsa ==>  one way anova 
varsayım sağlanmıyorsa ==> kruksal

yöntemleri kullanılır.

In [58]:
# dataset dahil edilmesi
df = sns.load_dataset("tips")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99000,1.01000,Female,No,Sun,Dinner,2
1,10.34000,1.66000,Male,No,Sun,Dinner,3
2,21.01000,3.50000,Male,No,Sun,Dinner,3
3,23.68000,3.31000,Male,No,Sun,Dinner,2
4,24.59000,3.61000,Female,No,Sun,Dinner,4


In [59]:
df.groupby("day")["total_bill"].mean()

day
Thur   17.68274
Fri    17.15158
Sat    20.44138
Sun    21.41000
Name: total_bill, dtype: float64

In [60]:
#1)
# hipotez kurulması
# grup ortalalamarı arasında fark yoktur.

#HO: m1=m2=m3=m4
#H!: ... fark vardır.

#2)
# varsayımların kontrolü
# normallik ve varyans homojenliği varsayımı


In [65]:
# normallik varsayımı 


# kategorik değişkenin sınıfları üzerinde normallik varsayımı sorguluyoruz.
for group in list(df["day"].unique()):
    pvalue = shapiro(df.loc [df["day"]==group,"total_bill"])[1]
    print(group,"p-value %.4f" % pvalue)
    
# p<0.05 | HO red edilir 
# normallik varsayımı sağlanmaz.


Sun p-value 0.0036
Sat p-value 0.0000
Thur p-value 0.0000
Fri p-value 0.0409


In [67]:
# Varyans homojenliği


t_test,pvalue = levene(df.loc [df["day"]=="Sun","total_bill"],
                               df.loc [df["day"]=="Sat","total_bill"],
                               df.loc [df["day"]=="Thur","total_bill"],
                               df.loc [df["day"]=="Fri","total_bill"])


print("T_test : %.4f \nP_value: %.4f" %(t_test,pvalue))


# HO red edilemez.
# Homojen varsayımı sağlanır.



T_test : 0.6654 
P_value: 0.5741


In [74]:
# Hipotez Testi ve p-value değeri yorumlanması 
# HO: Grup ortalamaları arasında istatistik olarak anlamlı fark yoktur.

t_test,pvalue = kruskal(df.loc[df["day"] == "Thur","total_bill"],
         df.loc[df["day"] == "Fri","total_bill"],
         df.loc[df["day"] == "Sat","total_bill"],
         df.loc[df["day"] == "Sun","total_bill"],
         )

print("T_test : %.4f \nP_value: %.4f" %(t_test,pvalue))

# p < 0.05 
# HO red edilir.
# Anlamlı bir farklılık vardır.


T_test : 10.4031 
P_value: 0.0154


In [73]:
# Varsayımın sağlandığı senorya için hipotez testi aşaması;

t_test,pvalue = f_oneway(df.loc[df["day"] == "Thur","total_bill"],
         df.loc[df["day"] == "Fri","total_bill"],
         df.loc[df["day"] == "Sat","total_bill"],
         df.loc[df["day"] == "Sun","total_bill"],
         )

print("T_test : %.4f \nP_value: %.4f" %(t_test,pvalue))

# ho red edilir. 
# anlamlı bir farklılık vardır.


T_test : 2.7675 
P_value: 0.0425


In [80]:
# yeni problem:
# Farklılık vardır ama farklılık hangi sınıftan kaynaklanmaktadır. ? 


from statsmodels.stats.multicomp import MultiComparison

comparison = MultiComparison(df["total_bill"],df["day"])
tukey = comparison.tukeyhsd(0.10)
print(tukey.summary())

Multiple Comparison of Means - Tukey HSD, FWER=0.10
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
   Fri    Sat   3.2898 0.4541 -1.8481 8.4277  False
   Fri    Sun   4.2584 0.2371 -0.9457 9.4626  False
   Fri   Thur   0.5312 0.9957 -4.7892 5.8515  False
   Sat    Sun   0.9686 0.8968  -2.217 4.1543  False
   Sat   Thur  -2.7586 0.2374 -6.1308 0.6135  False
   Sun   Thur  -3.7273 0.0668 -7.1995 -0.255   True
---------------------------------------------------
